In [ ]:
#importing required libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout
from tensorflow.keras.utils import image_dataset_from_directory as idfd
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
data_path = '/Users/omid/Desktop/data/'
train_data_path = '/Users/omid/Desktop/data/Train'
train_label_path = '/Users/omid/Desktop/data/Train_data_label.xlsx'
test_label_path = '/Users/omid/Desktop/data/Test_data_label.xlsx'
test_data_path = '/Users/omid/Desktop/data/Test/'

#reading train labels

train_labels = pd.read_excel(train_label_path)
test_labels = pd.read_excel(test_label_path)


In [ ]:
train_labels.head()

In [ ]:
#load images from directory into np array format

import os
from PIL import Image, ImageOps
    
def load_image(directory):
    images = []
    for x in train_labels.index:
        image = Image.open(os.path.join(directory, train_labels.loc[x, "Path"]))
        if image is not None:
            images.append(np.array(image))
        image.close()
    return images

def load_test(directory):
    images = []
    for x in os.listdir(directory):
        try:
            image = Image.open(os.path.join(directory, x))
        except:
            continue
        if image is not None:
            images.append(np.array(image))
        image.close()
    return images

def load_test_wlabel(directory):
    images = []
    for x in test_labels.index:
        image = Image.open(os.path.join(directory, test_labels.loc[x, "Path"]))
        if image is not None:
            images.append(np.array(image))
        image.close()
    return images

In [ ]:
train_images = load_image(data_path)
test_images = load_test(test_data_path)
ltest_images = load_test_wlabel(data_path)

In [ ]:
plt.imshow(train_images[5])

In [ ]:
#Resize all images to 30x30
def resize(train_images):
    for x in range(len(train_images)):
        im = Image.fromarray(train_images[x])
        im = im.resize((30, 30))
        train_images[x] = np.array(im)
        im.close()

In [ ]:
resize(train_images)
train_images[0].shape

In [ ]:
plt.imshow(train_images[0])

In [ ]:
#turns images to grayscale for grayscale CNN model
from skimage import color

def gray_scale(train):
    grays= []
    for x in range(len(train)):
        im = Image.fromarray(train[x])
        im = ImageOps.grayscale(im)
        t = np.array(im)
        grays.append(t)
        im.close()
    return grays

In [ ]:
#Saving the color and grayscale image array in np array format
color_img_arr = np.array(train_images)
grayscale_img_arr = np.array(gray_scale(train_images))

In [ ]:
#Target value
y = train_labels['ClassId'].values
print(y)

In [ ]:
#Plotting Bar Chart of Classes from train and test data by replacing numbers with actual class names
classes = ["Speed limit (20km/h)", "Speed limit (30km/h)", "Speed limit (50km/h)", "Speed limit (60km/h)", "Speed limit (70km/h)" 
, "Speed limit (80km/h)", "End of speed limit (80km/h)", "Speed limit (100km/h)", "Speed limit (120km/h)", "No passing", "No passing vehicle over 3.5 tons" 
, "Right-of-way at the intersection", "Priority road", "Yield", "Stop", "No vehicles", "Vehicle > 3.5 tons prohibited", "No entry", 
"General caution", "Dangerous curve left", "Dangerous curve right", "Double curve", "Bumpy road", "Slippery road", "Road narrows on the right", 
"Road work", "Traffic signals", "Pedestrians", "Children crossing", "Bicycles crossing", "Beware of ice/snow", "Wild animals crossing", 
"End speed + passing limits", "Turn right ahead", "Turn left ahead", "Ahead only", "Go straight or right", "Go straight or left", 
"Keep right", "Keep left", "Roundabout mandatory", "End of no passing", "End no passing vehicle > 3.5 tons"]

fig = plt.figure(figsize = (25,9))
vc = train_labels["ClassId"].value_counts().sort_index()
plt.bar(vc.index,vc.values)
class_legend = ('\n').join(f'{ids} - {classe}' for ids,classe in zip(vc.index,classes))
plt.text(45, 5, class_legend)
plt.subplots_adjust(right=.5)
plt.xticks(vc.index)
plt.xlabel("Classes")
plt.ylabel("Number of Images")
plt.title("Number of Images per Class in Training Data")
plt.show()

In [ ]:
fig = plt.figure(figsize = (25,9))
vc = test_labels["ClassId"].value_counts().sort_index()
plt.bar(vc.index,vc.values)
class_legend = ('\n').join(f'{ids} - {classe}' for ids,classe in zip(vc.index,classes))
plt.text(45, 5, class_legend)
plt.subplots_adjust(right=.5)
plt.xticks(vc.index)
plt.xlabel("Classes")
plt.ylabel("Number of Images")
plt.title("Number of Images per Class in Test Data")
plt.show()

In [ ]:
#Split Color Array Data into Train and Test
CX_train, CX_val, Cy_train, Cy_val = train_test_split(color_img_arr, y, random_state=42)

plt.imshow(CX_train[0])
print(Cy_train[0])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

#create Color model

model = Sequential()
#add model layers
model.add(Conv2D(32, kernel_size=(5,5), activation='relu', input_shape=(30,30,3)))
model.add(Conv2D(32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

early_stop = EarlyStopping(patience=2, monitor='val_loss')

In [ ]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
colors = model.fit(CX_train, Cy_train, epochs=5, validation_data=(CX_val, Cy_val), callbacks=[early_stop])

In [ ]:
#Plotting training and validation accuracy of Color Image CNN Model

plt.figure(0)
plt.plot(colors.history['accuracy'], label='training accuracy')
plt.plot(colors.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.figure(1)
plt.plot(colors.history['loss'], label='training loss')
plt.plot(colors.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#Evaluate with Test Data

resize(test_images)

CX_test = np.array(test_images)
Cy_test = test_labels['ClassId'].values

score = model.evaluate(CX_test, Cy_test)


In [ ]:
#Make Predictions with color CNN model

predictions = model.predict(CX_test)
pred_class = np.argmax(predictions, axis=1)
print(pred_class)

In [ ]:
plt.imshow(CX_test[0])

In [ ]:
#Evaluating Predictions
from sklearn.metrics import classification_report

print(classification_report(Cy_test, pred_class))

In [ ]:
GX_train, GX_val, Gy_train, Gy_val = train_test_split(grayscale_img_arr, y, random_state=42)

plt.imshow(GX_train[0], cmap='gray')
print(Gy_train[0])

In [ ]:
#create Grayscale model

gmodel = Sequential()
#add model layers
gmodel.add(Conv2D(32, kernel_size=(5,5), activation='relu', input_shape=(30,30,1)))
gmodel.add(Conv2D(32, kernel_size=(5,5), activation='relu'))
gmodel.add(MaxPool2D(pool_size=(2, 2)))
gmodel.add(Dropout(rate=0.25))
gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
gmodel.add(MaxPool2D(pool_size=(2, 2)))
gmodel.add(Dropout(rate=0.25))
gmodel.add(Flatten())
gmodel.add(Dense(256, activation='relu'))
gmodel.add(Dropout(rate=0.5))
gmodel.add(Dense(43, activation='softmax'))

early_stop = EarlyStopping(patience=2, monitor='val_loss')

In [ ]:
gmodel.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
gmodel.summary()

In [ ]:
gray = gmodel.fit(GX_train, Gy_train, epochs=5, validation_data=(GX_val, Gy_val), callbacks=[early_stop])

In [ ]:
#Plotting training and validation accuracy of Grayscale Image CNN Model

plt.figure(0)
plt.plot(gray.history['accuracy'], label='training accuracy')
plt.plot(gray.history['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.figure(1)
plt.plot(gray.history['loss'], label='training loss')
plt.plot(gray.history['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

In [ ]:
#Evaluate with Test Data

test_ig = gray_scale(test_images)

GX_test = np.array(test_ig)
Gy_test = test_labels['ClassId'].values

gscore = gmodel.evaluate(GX_test, Gy_test)

In [ ]:
#make predictions with Grayscale CNN model

predictions_gray = gmodel.predict(GX_test)
gpred_class = np.argmax(predictions_gray, axis=1)
print(gpred_class)

In [ ]:
plt.imshow(GX_test[2])

In [ ]:
#Evaluating Predictions
from sklearn.metrics import classification_report

print(classification_report(Gy_test, gpred_class))

In [ ]:
#Based on the performance and evaluation of both the color and grayscale model, we can say the grayscale model has a higher accuracy and lower loss 
#and therefore is the better model

#During both the fitting and evaluation phase the training, validation and test accuracy were higher for the grayscale model

#plot compares training and validation accuracy/loss of both to visualize that the grayscale model is better
plt.figure(0)
plt.plot(gray.history['accuracy'], label='grayscale training accuracy')
plt.plot(colors.history['accuracy'], label='color training accuracy')
plt.plot(gray.history['val_accuracy'], label='grayscale val accuracy')
plt.plot(colors.history['val_accuracy'], label='color val accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()
plt.figure(1)
plt.plot(gray.history['loss'], label='grayscale training loss')
plt.plot(colors.history['loss'], label='color training loss')
plt.plot(gray.history['val_loss'], label='grayscale val loss')
plt.plot(colors.history['val_loss'], label='color val loss')
plt.title('Loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()